## Model: Whisper

In [4]:
!pip install -U openai-whisper --quiet
!apt-get install ffmpeg --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import whisper
import os

# Load Whisper model
model = whisper.load_model("small")

# Paths
audio_folder = "/content/drive/MyDrive/audio_files/"
output_folder = "/content/drive/MyDrive/ServiceNow_Audio_Transcripts/"
os.makedirs(output_folder, exist_ok=True)

# Transcribe only if transcript file doesn't exist
for filename in os.listdir(audio_folder):
    if filename.endswith((".mp3", ".wav", ".m4a")):
        base_name = os.path.splitext(filename)[0]
        transcript_path = os.path.join(output_folder, base_name + ".txt")

        if os.path.exists(transcript_path) and os.path.getsize(transcript_path) > 0:
            print(f"Transcript already exists for: {filename} — Skipping.")
            continue

        print(f"Transcribing: {filename}")
        audio_path = os.path.join(audio_folder, filename)
        result = model.transcribe(audio_path)

        with open(transcript_path, "w", encoding='utf-8') as f:
            f.write(result['text'])

100%|███████████████████████████████████████| 461M/461M [00:10<00:00, 45.2MiB/s]


Transcript already exists for: ThW6lPyYgYk.mp3 — Skipping.
Transcript already exists for: XUKTyE2YtHc&list=PLh-u-epknspBswAAKG0EfPHyV6gcVVOhK.mp3 — Skipping.
Transcript already exists for: Rx65d0ofz8I.mp3 — Skipping.
Transcript already exists for: WyQTP0AA1VU.mp3 — Skipping.
Transcript already exists for: ZYJqkxGrNiI.mp3 — Skipping.
Transcript already exists for: E6m8UuVhIzw.mp3 — Skipping.
Transcript already exists for: K6z4c256gzI.mp3 — Skipping.
Transcript already exists for: KSWNDuKn9t0.mp3 — Skipping.
Transcript already exists for: 7WJ6lmxa1WQ.mp3 — Skipping.
Transcript already exists for: fqB-NcZmqXo.mp3 — Skipping.
Transcript already exists for: eFMeZto6yMg.mp3 — Skipping.
Transcript already exists for: _2IG1CX2y6g.mp3 — Skipping.
Transcript already exists for: it1hcs5S1ks&t=27s.mp3 — Skipping.
Transcript already exists for: a0fllfx_fmg&list=PLkGSnjw5y2U407_1UQQaVVrD13-MFi5ia&index=22.mp3 — Skipping.
Transcript already exists for: kQV6g8Vbbfc.mp3 — Skipping.
Transcript already e

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

transcript_folder = "/content/drive/MyDrive/ServiceNow_Audio_Transcripts/"
all_chunks = []

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# Intro filter configuration
intro_phrases = [
    "welcome", "in this video", "today we're", "this tutorial",
    "thank you for joining", "let’s get started","Hello", "Hey", "Hi", "Good morning", "Good afternoon", "Good evening"
]

def is_intro(text):
    return any(phrase in text.lower() for phrase in intro_phrases)

# Load and filter transcript chunks
for filename in os.listdir(transcript_folder):
    if filename.endswith(".txt"):
        with open(os.path.join(transcript_folder, filename), "r", encoding="utf-8") as f:
            raw_text = f.read()
            chunks = splitter.split_text(raw_text)
            filtered_chunks = [chunk for chunk in chunks if not is_intro(chunk)]
            all_chunks.extend(filtered_chunks)

In [11]:
!pip install faiss-cpu sentence-transformers langchain --quiet
!pip install -U langchain-community --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.4 MB/s eta 0:00:00


In [12]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create vector store
faiss_store = FAISS.from_texts(all_chunks, embedding=embedding_model)


/tmp/ipython-input-12-2798286620.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
!pip install -U langchain-openai
from langchain_openai import OpenAI
import os
from google.colab import userdata
llm = OpenAI(
    temperature=0,
    openai_api_key=userdata.get('OPENAI_API_KEY')
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 3.1 MB/s eta 0:00:00


In [14]:
from langchain.chains import RetrievalQA
retriever = faiss_store.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Ask a question
question = "What are the steps to configure incident management in ServiceNow?"
answer = qa_chain.run(question)
print("Answer:", answer)

/tmp/ipython-input-14-18830977.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(question)


Answer: 
1. Set up a ServiceNow instance: The first step is to set up a ServiceNow instance and configure it according to your organization's needs.

2. Define incident management roles: Identify the roles and responsibilities of the team members who will be involved in incident management. This can include incident managers, service desk agents, and other IT staff.

3. Configure incident management settings: In the ServiceNow instance, go to the Incident Management application and configure the settings according to your organization's needs. This includes setting up incident categories, priorities, and SLAs.

4. Create incident templates: Create templates for common incidents to help streamline the incident creation process.

5. Set up incident assignment rules: Configure rules to automatically assign incidents to the appropriate team or individual based on the incident category, priority, or other criteria.

6. Configure notifications: Set up notifications to keep stakeholders infor

In [15]:
question = "How does ServiceNow use AI to improve incident management?"
answer = qa_chain.run(question)
print("Answer:", answer)

Answer:  ServiceNow uses agentic AI, which is a framework built on the Now platform. This AI is native to the platform and is constantly learning and making decisions based on data from various sources, without being prompted. This allows for more efficient and accurate incident management, as the AI can handle a large volume of data and make decisions based on specific criteria. Additionally, ServiceNow can train the AI specifically for their platform, making it even more effective in managing incidents.


In [16]:
question = "What is predictive intelligence in ServiceNow?"
answer = qa_chain.run(question)
print("Answer:", answer)

Answer:  Predictive intelligence in ServiceNow is a feature that requires the creation of three different models to predict three different fields. It is different from task intelligence, which uses a single model and is purpose-built for business users.


In [17]:
question = "Explain how machine learning models are trained in ServiceNow Predictive Intelligence."
answer = qa_chain.run(question)
print("Answer:", answer)

Answer:  In ServiceNow Predictive Intelligence, machine learning models are trained by creating three different models to predict three different fields. This requires specific training off of ServiceNow workflows, behaviors, and architecture. However, in Task Intelligence, a single model is applicable to multiple fields and is trained using deep learning and long text support models. This allows for better results as the model can take into account both the short description and description fields. Additionally, agentic AI, which is a type of machine learning used in ServiceNow, is less dependent on the size and variety of information it is trained on. It can be trained on all the world's knowledge of workflow automation, but can also be trained specifically to act like ServiceNow.


In [18]:
question = "What AI features are available in the ServiceNow platform?"
answer = qa_chain.run(question)
print("Answer:", answer)

Answer:  The ServiceNow platform offers a native agentic AI framework, which includes AI agents for automating tasks and correlating events. This feature is available to customers without the need to learn complex AI tools.
